
# 🧩 Bloc 2 — Sessió 3  
## Normalització, correlació i visualització exploratòria



## 1. Preparació de l'entorn

En aquesta sessió treballarem amb:

- `pandas` per a la manipulació de dades
- `numpy` per a càlcul numèric
- `matplotlib` i `seaborn` per a la visualització
- eines de *scikit-learn* per a l'escalat de dades

> **Nota:** Si estàs a un entorn nou, assegura't de tenir instal·lades aquestes llibreries.


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

sns.set()
print("Llibreries carregades correctament.")



## 2. Carrega o creació del conjunt de dades

En un projecte real, carregaríem un fitxer `.csv` amb dades proporcionades.  
Per assegurar que el notebook és autosuficient, aquí **crearem un dataset sintètic** que simula:

- `edat` (anys)
- `altura` (cm)
- `pes` (kg)
- `ingressos` (sou mensual aproximat)
- `target` (variable objectiu fictícia, per exemple: nivell de benestar de 0 a 10)


In [ ]:

np.random.seed(42)

n = 300

edat = np.random.normal(loc=40, scale=12, size=n).clip(18, 70)
altura = np.random.normal(loc=170, scale=10, size=n).clip(145, 200)
pes = np.random.normal(loc=70, scale=12, size=n).clip(45, 120)

# Ingressos lleugerament correlacionats amb edat i altura
ingressos = (edat * 30 + altura * 5 + np.random.normal(0, 300, size=n)).clip(600, 6000)

# Variable objectiu: nivell de benestar fictici (0-10)
target = (0.02 * ingressos + 0.1 * (altura - 160) - 0.05 * (edat - 40)
          + np.random.normal(0, 3, size=n))
target = target.clip(0, 10)

df = pd.DataFrame({
    "edat": edat.round(0),
    "altura": altura.round(1),
    "pes": pes.round(1),
    "ingressos": ingressos.round(0),
    "target": target.round(2)
})

df.head()



### 2.1. Exploració inicial de les dades


In [ ]:

df.describe()


In [ ]:

df.info()



## 3. Normalització i estandardització

Treballarem amb dues transformacions habituals:

- **Normalització min-max**  
  $x' = \frac{x - \min(x)}{\max(x) - \min(x)}$

- **Estandardització (z-score)**  
  $z = \frac{x - \mu}{\sigma}$

Aquestes transformacions són especialment útils per a algoritmes sensibles a l'escala, com ara KNN, regressió logística o xarxes neuronals.


In [ ]:

features = ["edat", "altura", "pes", "ingressos"]

scaler_minmax = MinMaxScaler()
scaler_standard = StandardScaler()

df_minmax = df.copy()
df_standard = df.copy()

df_minmax[[f + "_minmax" for f in features]] = scaler_minmax.fit_transform(df[features])
df_standard[[f + "_std" for f in features]] = scaler_standard.fit_transform(df[features])

df_minmax.head()



### 3.1. Comparació visual abans i després de l'escalat


In [ ]:

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(df["ingressos"], bins=20)
axes[0].set_title("Ingressos - escala original")

axes[1].hist(df_minmax["ingressos_minmax"], bins=20)
axes[1].set_title("Ingressos - min-max")

plt.tight_layout()
plt.show()



## 4. Correlació entre variables

La correlació de Pearson mesura la força i la direcció d'una relació **lineal** entre dues variables numèriques.

- Valors propers a **1** → correlació positiva forta  
- Valors propers a **-1** → correlació negativa forta  
- Valors propers a **0** → poca o cap correlació lineal


In [ ]:

corr_matrix = df.corr(numeric_only=True)
corr_matrix


In [ ]:

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Matriu de correlació")
plt.show()



> **Preguntes per reflexionar:**
>
> - Quines variables estan més relacionades amb `target`?
> - Té sentit, des del punt de vista del món real, la correlació observada?
> - Hi ha alguna correlació que sembli "sospitosa" o casual?



## 5. Visualització exploratòria (EDA)

Fem alguns gràfics per entendre millor les dades: distribucions, relacions i possibles valors atípics.


In [ ]:

# Histogrames
df[["edat", "altura", "pes", "ingressos"]].hist(bins=20, figsize=(10, 8))
plt.suptitle("Histogrames de les variables numèriques")
plt.tight_layout()
plt.show()


In [ ]:

# Scatter plot: altura vs pes
plt.figure(figsize=(6, 4))
sns.scatterplot(x="altura", y="pes", data=df)
plt.title("Relació entre altura i pes")
plt.show()


In [ ]:

# Boxplot: ingressos
plt.figure(figsize=(6, 4))
sns.boxplot(x=df["ingressos"])
plt.title("Distribució d'ingressos i possibles outliers")
plt.show()


In [ ]:

# Pairplot d'algunes variables
sns.pairplot(df[["edat", "altura", "pes", "ingressos", "target"]])
plt.suptitle("Pairplot de variables clau", y=1.02)
plt.show()



> **Activitats suggerides:**
>
> 1. Identifica una variable amb distribució asimètrica (*skewed*).  
> 2. Proposa una transformació (per exemple, logarítmica) per millorar-ne la distribució.  
> 3. Indica dues variables que semblin especialment rellevants per predir `target`.
